In [162]:
#import statements
import numpy as np
import pandas as pd
import json
import requests

from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

### Data Loading

Data sourced from https://www.kaggle.com/datasets/nikdavis/steam-store-games?select=steamspy_tag_data.csv

The author of the dataset documented their process of using Steam APIs here:
https://nik-davis.github.io/posts/2019/steam-data-collection/

The dataset is stored under a data folder in my local repository. 

The filepath is ./data/Steam_store_data

There are 6 CSV files in the dataset.

In [2]:
!ls

README.md
Steam API Calls.txt
SteamWebAPIKey.txt
data
notebook.ipynb
scratch_notebook.ipynb


In [3]:
steam_df = pd.read_csv('./data/Steam_store_data/steam.csv')

In [4]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

### Exploring the data

In [5]:
steam_df.shape

(27075, 18)

There are 27,705 rows - each representing a game in Steam's store - and 18 columns.

Let's go through each column one-by-one:

In [6]:
#unique id value identifier for game
steam_df['appid']

0             10
1             20
2             30
3             40
4             50
          ...   
27070    1065230
27071    1065570
27072    1065650
27073    1066700
27074    1069460
Name: appid, Length: 27075, dtype: int64

In [7]:
#name of game
steam_df['name']

0                    Counter-Strike
1             Team Fortress Classic
2                     Day of Defeat
3                Deathmatch Classic
4         Half-Life: Opposing Force
                    ...            
27070               Room of Pandora
27071                     Cyber Gun
27072              Super Star Blast
27073    New Yankee 7: Deer Hunters
27074                     Rune Lord
Name: name, Length: 27075, dtype: object

In [8]:
#date of release (YY-MM-DD)
steam_df['release_date']

0        2000-11-01
1        1999-04-01
2        2003-05-01
3        2001-06-01
4        1999-11-01
            ...    
27070    2019-04-24
27071    2019-04-23
27072    2019-04-24
27073    2019-04-17
27074    2019-04-24
Name: release_date, Length: 27075, dtype: object

In [9]:
#release_date is stored as a string
type(steam_df['release_date'][0])

str

In [131]:
steam_df['release_date'].max()

'2019-05-01'

In [10]:
steam_df['english']

0        1
1        1
2        1
3        1
4        1
        ..
27070    1
27071    1
27072    1
27073    1
27074    1
Name: english, Length: 27075, dtype: int64

In [11]:
steam_df['english'].value_counts()

1    26564
0      511
Name: english, dtype: int64

There are 26,564 English games and 511 non-English games.

In [12]:
#name of game's developer
steam_df['developer']

0                     Valve
1                     Valve
2                     Valve
3                     Valve
4          Gearbox Software
                ...        
27070           SHEN JIAWEI
27071        Semyon Maximov
27072           EntwicklerX
27073    Yustas Game Studio
27074      Adept Studios GD
Name: developer, Length: 27075, dtype: object

In [13]:
#name of game's publisher
steam_df['publisher']

0                       Valve
1                       Valve
2                       Valve
3                       Valve
4                       Valve
                 ...         
27070             SHEN JIAWEI
27071        BekkerDev Studio
27072             EntwicklerX
27073    Alawar Entertainment
27074    Alawar Entertainment
Name: publisher, Length: 27075, dtype: object

In [14]:
#platform availability for game
steam_df['platforms']

0        windows;mac;linux
1        windows;mac;linux
2        windows;mac;linux
3        windows;mac;linux
4        windows;mac;linux
               ...        
27070              windows
27071              windows
27072              windows
27073          windows;mac
27074          windows;mac
Name: platforms, Length: 27075, dtype: object

In [15]:
steam_df['platforms'].value_counts()

windows              18398
windows;mac;linux     4623
windows;mac           3439
windows;linux          610
mac                      3
mac;linux                1
linux                    1
Name: platforms, dtype: int64

In [16]:
steam_df[steam_df['platforms'] == 'mac']

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
1413,214630,Call of Duty: Black Ops - Mac Edition,2012-09-27,1,Aspyr,Aspyr,mac,18,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Action;Zombies;Multiplayer,68,168,105,0,0,50000-100000,15.49
12479,569050,Paul Pixel - The Awakening,2017-01-09,1,Xoron GmbH,Xoron GmbH,mac,0,Single-player,Adventure;Indie,Adventure;Indie;Point & Click,0,5,0,0,0,0-20000,2.89
16662,694180,MobileZombie,2017-10-13,1,YIMING ZHANG,YIMING ZHANG,mac,0,Single-player;Partial Controller Support,Adventure;Casual;Free to Play;Indie,Free to Play;Adventure;Indie,0,14,11,0,0,0-20000,0.00


In [17]:
#age-requirement for game
steam_df['required_age']

0        0
1        0
2        0
3        0
4        0
        ..
27070    0
27071    0
27072    0
27073    0
27074    0
Name: required_age, Length: 27075, dtype: int64

In [18]:
steam_df['required_age'].value_counts()

0     26479
18      308
16      192
12       73
7        12
3        11
Name: required_age, dtype: int64

In [19]:
steam_df['categories']

0        Multi-player;Online Multi-Player;Local Multi-P...
1        Multi-player;Online Multi-Player;Local Multi-P...
2                    Multi-player;Valve Anti-Cheat enabled
3        Multi-player;Online Multi-Player;Local Multi-P...
4        Single-player;Multi-player;Valve Anti-Cheat en...
                               ...                        
27070                     Single-player;Steam Achievements
27071                                        Single-player
27072    Single-player;Multi-player;Co-op;Shared/Split ...
27073                            Single-player;Steam Cloud
27074                            Single-player;Steam Cloud
Name: categories, Length: 27075, dtype: object

In [20]:
#there are a lot of combinations for categories
#might have to clean this up separately to see pull out the most meaningful categories
steam_df['categories'].value_counts()

Single-player                                                                                                                                                                          6110
Single-player;Steam Achievements                                                                                                                                                       2334
Single-player;Steam Achievements;Steam Trading Cards                                                                                                                                    848
Single-player;Partial Controller Support                                                                                                                                                804
Single-player;Steam Trading Cards                                                                                                                                                       792
                                                            

In [21]:
def extract_category(df, column):
    '''
    Returns a dictionary containing categories and their counts from the dataframe
    Categories and genres are stored separated by a semicolon
    '''
    category_dict = {}
    for tags in df[column]:
        split_tags = tags.split(';')
        for x in split_tags:
            if x not in category_dict:
                category_dict[x] = 1
            else:
                category_dict[x] += 1
    return category_dict

In [22]:
categories = extract_category(steam_df, 'categories')
categories

{'Multi-player': 3974,
 'Online Multi-Player': 2487,
 'Local Multi-Player': 1615,
 'Valve Anti-Cheat enabled': 94,
 'Single-player': 25678,
 'Steam Cloud': 7219,
 'Steam Achievements': 14130,
 'Steam Trading Cards': 7918,
 'Captions available': 721,
 'Partial Controller Support': 4234,
 'Includes Source SDK': 35,
 'Cross-Platform Multiplayer': 1081,
 'Stats': 1878,
 'Commentary available': 144,
 'Includes level editor': 1036,
 'Steam Workshop': 897,
 'In-App Purchases': 690,
 'Co-op': 1721,
 'Full controller support': 5695,
 'Steam Leaderboards': 3439,
 'SteamVR Collectibles': 40,
 'Online Co-op': 1071,
 'Shared/Split Screen': 2152,
 'Local Co-op': 1059,
 'MMO': 421,
 'VR Support': 231,
 'Mods': 2,
 'Mods (require HL2)': 1,
 'Steam Turn Notifications': 63}

In [23]:
#maybe subset categories based on number of players
categories_players = ['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Single-player', 'Co-op', 'Online Co-op', 'Shared/Split Screen', 'Local Co-op']
subset_categories = [x for x in categories if x in categories_players]
subset_categories

['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Single-player',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op']

In [24]:
steam_df['genres'].value_counts()

Action;Indie                                                              1852
Casual;Indie                                                              1482
Action;Adventure;Indie                                                    1229
Adventure;Indie                                                           1170
Action;Casual;Indie                                                       1004
                                                                          ... 
Action;Casual;Free to Play;Indie;Massively Multiplayer;Simulation            1
Sexual Content;Nudity;Action                                                 1
Sexual Content;Nudity;Violent;Gore;Adventure;Indie                           1
Sexual Content;Violent;Gore;Action;Adventure;Casual;Indie;RPG;Strategy       1
Action;Adventure;Massively Multiplayer;RPG;Strategy;Early Access             1
Name: genres, Length: 1552, dtype: int64

In [25]:
genre_dict = extract_category(steam_df, 'genres')
genre_dict

{'Action': 11903,
 'Free to Play': 1704,
 'Strategy': 5247,
 'Adventure': 10032,
 'Indie': 19421,
 'RPG': 4311,
 'Animation & Modeling': 79,
 'Video Production': 38,
 'Casual': 10210,
 'Simulation': 5194,
 'Racing': 1024,
 'Violent': 843,
 'Massively Multiplayer': 723,
 'Nudity': 266,
 'Sports': 1322,
 'Early Access': 2954,
 'Gore': 537,
 'Utilities': 146,
 'Design & Illustration': 87,
 'Web Publishing': 28,
 'Education': 51,
 'Software Training': 31,
 'Sexual Content': 245,
 'Audio Production': 29,
 'Game Development': 17,
 'Photo Editing': 12,
 'Accounting': 6,
 'Documentary': 1,
 'Tutorial': 1}

Some of these items aren't games; some are developer tools or asset packs. I'll focus on popular genres to deliver generalized results. 

In [26]:
genres = list(genre_dict.keys())
genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Animation & Modeling',
 'Video Production',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Nudity',
 'Sports',
 'Early Access',
 'Gore',
 'Utilities',
 'Design & Illustration',
 'Web Publishing',
 'Education',
 'Software Training',
 'Sexual Content',
 'Audio Production',
 'Game Development',
 'Photo Editing',
 'Accounting',
 'Documentary',
 'Tutorial']

In [27]:
genres_to_drop = ['Animation & Modeling', 'Video Production', 'Nudity', 'Gore', 'Utilities', 'Design & Illustration', 'Web Publishing', 'Education', 'Software Training', 'Sexual Content', 'Audio Production', 'Game Development', 'Photo Editing', 'Accounting', 'Documentary', 'Tutorial']
clean_genres = [x for x in genres if x not in genres_to_drop]
clean_genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Sports',
 'Early Access']

In [28]:
print(len(genres))
print(len(genres_to_drop))
print(len(clean_genres))

29
16
13


In [29]:
#content genres pulled using SteamSpy API
steam_df['steamspy_tags']

0              Action;FPS;Multiplayer
1              Action;FPS;Multiplayer
2        FPS;World War II;Multiplayer
3              Action;FPS;Multiplayer
4                   FPS;Action;Sci-fi
                     ...             
27070          Adventure;Indie;Casual
27071          Action;Indie;Adventure
27072             Action;Indie;Casual
27073          Indie;Casual;Adventure
27074          Indie;Casual;Adventure
Name: steamspy_tags, Length: 27075, dtype: object

In [30]:
#content genres
steam_df['steamspy_tags'].value_counts()

Action;Indie;Casual               845
Action;Adventure;Indie            714
Early Access;Action;Indie         507
Adventure;Indie;Casual            442
Indie;Casual                      378
                                 ... 
RPG;Sexual Content;Nudity           1
Indie;Difficult;Platformer          1
Early Access;Indie;Beat 'em up      1
Sexual Content;Casual;RPG           1
Indie;Puzzle-Platformer;Co-op       1
Name: steamspy_tags, Length: 6423, dtype: int64

In [31]:
steamspy_tags = extract_category(steam_df, 'steamspy_tags')
steamspy_tags

{'Action': 10322,
 'FPS': 405,
 'Multiplayer': 405,
 'World War II': 130,
 'Sci-fi': 157,
 'Classic': 170,
 'Singleplayer': 161,
 'Puzzle': 1116,
 'First-Person': 91,
 'Free to Play': 1662,
 'Zombies': 158,
 'Co-op': 120,
 'MOBA': 13,
 'Strategy': 4173,
 'Shooter': 101,
 'Indie': 16232,
 'Fighting': 117,
 'Cyberpunk': 58,
 'RTS': 174,
 'Hacking': 20,
 'Simulation': 3284,
 'Naval': 39,
 '4X': 37,
 'Turn-Based Strategy': 103,
 'Fantasy': 115,
 'Tanks': 21,
 'Space': 225,
 'RPG': 2785,
 'Animation & Modeling': 54,
 'Video Production': 31,
 'Open World': 243,
 'Competitive': 12,
 'Top-Down Shooter': 27,
 'Casual': 8205,
 'Vampire': 17,
 'Cult Classic': 7,
 'Western': 23,
 'Racing': 765,
 'Destruction': 21,
 'Offroad': 17,
 'Adventure': 7770,
 'Match 3': 119,
 'Arcade': 203,
 'Hidden Object': 460,
 'Tower Defense': 168,
 'Sniper': 8,
 'Stealth': 93,
 'Base-Building': 26,
 'Villain Protagonist': 9,
 'Story Rich': 159,
 'Female Protagonist': 199,
 'Platformer': 648,
 'Comedy': 89,
 'Turn-Base

In [32]:
len(steamspy_tags)

339

In [33]:
for tag in sorted(steamspy_tags, key=steamspy_tags.get, reverse=True):
    print(tag, steamspy_tags[tag])
    
#maybe remove counts below 30, if you're really into those categories, you can browse those yourself

Indie 16232
Action 10322
Casual 8205
Adventure 7770
Strategy 4173
Simulation 3284
Early Access 2967
RPG 2785
Free to Play 1662
Puzzle 1116
VR 961
Sports 781
Racing 765
Platformer 648
Nudity 570
Sexual Content 557
Visual Novel 546
Violent 545
Anime 538
Horror 493
Point & Click 492
Gore 476
Hidden Object 460
FPS 405
Multiplayer 405
Massively Multiplayer 377
Pixel Graphics 249
Shoot 'Em Up 245
Open World 243
Survival 235
Space 225
Arcade 203
Female Protagonist 199
RPGMaker 192
RTS 174
Turn-Based 171
Classic 170
Tower Defense 168
Card Game 164
2D 163
Singleplayer 161
Story Rich 159
Zombies 158
Sci-fi 157
Utilities 138
World War II 130
Great Soundtrack 125
Co-op 120
Match 3 119
Fighting 117
Management 116
Fantasy 115
Retro 114
Music 105
Board Game 105
Rogue-like 104
Turn-Based Strategy 103
Shooter 101
Bullet Hell 96
Sandbox 94
JRPG 94
Stealth 93
First-Person 91
Comedy 89
Family Friendly 86
Atmospheric 85
Difficult 85
Education 84
Hack and Slash 83
Local Multiplayer 80
Design & Illustration 

In [34]:
#number of achievements available
steam_df['achievements']

0         0
1         0
2         0
3         0
4         0
         ..
27070     7
27071     0
27072    24
27073     0
27074     0
Name: achievements, Length: 27075, dtype: int64

In [35]:
steam_df['achievements'].value_counts()

0       11864
10        679
12        618
20        574
15        490
        ...  
184         1
168         1
1487        1
319         1
4094        1
Name: achievements, Length: 410, dtype: int64

In [36]:
#is achievements a meaningful consideration for buyers?
steam_df[steam_df['achievements']==4094]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
13740,604490,Running Through Russia,2017-05-21,1,Spell Helix,Spell Helix,windows;mac;linux,0,Single-player;Steam Achievements,Adventure;Casual;Indie;Early Access,Early Access;Indie;Casual,4094,751,335,22,22,50000-100000,0.79


In [37]:
steam_df['positive_ratings']

0        124534
1          3318
2          3416
3          1273
4          5250
          ...  
27070         3
27071         8
27072         0
27073         2
27074         4
Name: positive_ratings, Length: 27075, dtype: int64

In [38]:
steam_df['negative_ratings']

0        3339
1         633
2         398
3         267
4         288
         ... 
27070       0
27071       1
27072       1
27073       0
27074       0
Name: negative_ratings, Length: 27075, dtype: int64

In [39]:
no_ratings = steam_df[ (steam_df['positive_ratings'] == 0) &  (steam_df['negative_ratings'] == 0) ]
no_ratings
#every game in this dataset has at least 1 rating
#there shouldn't be any divide-by-0 errors if I create a % positive/% negative ratings feature

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price


In [40]:
steam_df['total_ratings'] = steam_df['positive_ratings'] + steam_df['negative_ratings']
steam_df['total_ratings']

0        127873
1          3951
2          3814
3          1540
4          5538
          ...  
27070         3
27071         9
27072         1
27073         2
27074         4
Name: total_ratings, Length: 27075, dtype: int64

In [41]:
steam_df['percent_positive'] = steam_df['positive_ratings'] / steam_df['total_ratings']
steam_df['percent_positive']

0        0.973888
1        0.839787
2        0.895648
3        0.826623
4        0.947996
           ...   
27070    1.000000
27071    0.888889
27072    0.000000
27073    1.000000
27074    1.000000
Name: percent_positive, Length: 27075, dtype: float64

In [42]:
#average user playtime (minutes)
steam_df['average_playtime']

0        17612
1          277
2          187
3          258
4          624
         ...  
27070        0
27071        0
27072        0
27073        0
27074        0
Name: average_playtime, Length: 27075, dtype: int64

In [43]:
steam_df[steam_df['average_playtime'] == 0]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,percent_positive
26,1002,Rag Doll Kung Fu,2005-10-12,1,Mark Healey,Mark Healey,windows,0,Single-player;Multi-player,Indie,Indie;Fighting;Multiplayer,0,40,17,0,0,20000-50000,5.99,57,0.701754
29,1300,SiN Episodes: Emergence,2006-05-10,1,Ritual Entertainment,Ritual Entertainment,windows,0,Single-player;Stats,Action,Action;FPS;Cyberpunk,0,468,61,0,0,100000-200000,7.19,529,0.884688
34,1600,Dangerous Waters,2006-02-07,1,Sonalysts,Strategy First,windows,0,Single-player;Multi-player,Strategy,Strategy;Simulation;Naval,0,140,44,0,0,50000-100000,22.99,184,0.760870
35,1610,Space Empires IV Deluxe,2006-02-07,1,Malfador Machinations,Strategy First,windows,0,Single-player;Multi-player,Strategy,Strategy;4X;Sci-fi,0,112,26,0,0,50000-100000,6.99,138,0.811594
36,1630,Disciples II: Rise of the Elves,2006-07-06,1,Strategy First,Strategy First,windows,0,Single-player;Multi-player;Co-op,Strategy,Strategy;Turn-Based Strategy;Fantasy,0,451,108,0,0,100000-200000,4.99,559,0.806798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09,3,1.000000
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69,9,0.888889
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99,1,0.000000
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19,2,1.000000


In [44]:
no_playtime = steam_df[steam_df['average_playtime'] == 0]
no_playtime.sort_values(by=['owners'], ascending=False)
no_playtime.shape

(20905, 20)

In [45]:
steam_df['median_playtime'].value_counts()

0       20905
1         155
3          72
2          52
9          48
        ...  
636         1
684         1
4164        1
748         1
6061        1
Name: median_playtime, Length: 1312, dtype: int64

In [46]:
#number of users who own this game, given as a range
steam_df['owners']

0        10000000-20000000
1         5000000-10000000
2         5000000-10000000
3         5000000-10000000
4         5000000-10000000
               ...        
27070              0-20000
27071              0-20000
27072              0-20000
27073              0-20000
27074              0-20000
Name: owners, Length: 27075, dtype: object

In [47]:
#price of game in USD
steam_df['price']

0        7.19
1        3.99
2        3.99
3        3.99
4        3.99
         ... 
27070    2.09
27071    1.69
27072    3.99
27073    5.19
27074    5.19
Name: price, Length: 27075, dtype: float64

In [48]:
steam_df['price'].value_counts()

3.99     3211
0.79     2892
0.00     2560
6.99     2050
7.19     1304
         ... 
7.42        1
3.03        1
6.10        1
11.75       1
4.70        1
Name: price, Length: 282, dtype: int64

In [49]:
steam_df.iloc[0]

appid                                                              10
name                                                   Counter-Strike
release_date                                               2000-11-01
english                                                             1
developer                                                       Valve
publisher                                                       Valve
platforms                                           windows;mac;linux
required_age                                                        0
categories          Multi-player;Online Multi-Player;Local Multi-P...
genres                                                         Action
steamspy_tags                                  Action;FPS;Multiplayer
achievements                                                        0
positive_ratings                                               124534
negative_ratings                                                 3339
average_playtime    

## Dataframe with Encoded Values

In [140]:
steam_df.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price',
       'total_ratings', 'percent_positive'],
      dtype='object')

In [155]:
#english games only
encoded_df = steam_df[steam_df['english'] == 1]
encoded_df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,percent_positive
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,127873,0.973888
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,3951,0.839787
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,3814,0.895648
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,1540,0.826623
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,5538,0.947996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09,3,1.000000
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69,9,0.888889
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99,1,0.000000
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19,2,1.000000


In [156]:
#dropping unnecessary columns
columns_to_drop=['release_date', 'developer', 'publisher', 'achievements', 'required_age', 'positive_ratings', 'negative_ratings',
                'average_playtime', 'median_playtime', 'owners', 'price', 'total_ratings', 'percent_positive']
encoded_df = encoded_df.drop(columns=columns_to_drop, axis=1)
encoded_df

,appid,name,english,platforms,categories,genres,steamspy_tags
0,10,Counter-Strike,1,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
1,20,Team Fortress Classic,1,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
2,30,Day of Defeat,1,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,1,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
4,50,Half-Life: Opposing Force,1,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi
...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,1,windows,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual
27071,1065570,Cyber Gun,1,windows,Single-player,Action;Adventure;Indie,Action;Indie;Adventure
27072,1065650,Super Star Blast,1,windows,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual
27073,1066700,New Yankee 7: Deer Hunters,1,windows;mac,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure


In [157]:
encoded_df['platforms'] = encoded_df['platforms'].str.split(';')
encoded_df['categories'] = encoded_df['categories'].str.split(';')
encoded_df['genres'] = encoded_df['genres'].str.split(';')
encoded_df['steamspy_tags'] = encoded_df['steamspy_tags'].str.split(';')

In [158]:
encoded_df

,appid,name,english,platforms,categories,genres,steamspy_tags
0,10,Counter-Strike,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
1,20,Team Fortress Classic,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
2,30,Day of Defeat,1,"[windows, mac, linux]","[Multi-player, Valve Anti-Cheat enabled]",[Action],"[FPS, World War II, Multiplayer]"
3,40,Deathmatch Classic,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
4,50,Half-Life: Opposing Force,1,"[windows, mac, linux]","[Single-player, Multi-player, Valve Anti-Cheat...",[Action],"[FPS, Action, Sci-fi]"
...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,1,[windows],"[Single-player, Steam Achievements]","[Adventure, Casual, Indie]","[Adventure, Indie, Casual]"
27071,1065570,Cyber Gun,1,[windows],[Single-player],"[Action, Adventure, Indie]","[Action, Indie, Adventure]"
27072,1065650,Super Star Blast,1,[windows],"[Single-player, Multi-player, Co-op, Shared/Sp...","[Action, Casual, Indie]","[Action, Indie, Casual]"
27073,1066700,New Yankee 7: Deer Hunters,1,"[windows, mac]","[Single-player, Steam Cloud]","[Adventure, Casual, Indie]","[Indie, Casual, Adventure]"


In [176]:
encoded_2 = encoded_df

In [177]:
mlb = MultiLabelBinarizer()
mlb_platforms = mlb.fit_transform(encoded_2['platforms'])
mlb_platforms

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       ...,
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 1]])

In [183]:
mlb_platforms.shape

(26564, 3)

In [179]:
mlb_df = pd.DataFrame(mlb_platforms, encoded_2.index, mlb.classes_)
mlb_df

,linux,mac,windows
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
27070,0,0,1
27071,0,0,1
27072,0,0,1
27073,0,1,1


In [180]:
encoded_2

,appid,name,english,platforms,categories,genres,steamspy_tags
0,10,Counter-Strike,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
1,20,Team Fortress Classic,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
2,30,Day of Defeat,1,"[windows, mac, linux]","[Multi-player, Valve Anti-Cheat enabled]",[Action],"[FPS, World War II, Multiplayer]"
3,40,Deathmatch Classic,1,"[windows, mac, linux]","[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]"
4,50,Half-Life: Opposing Force,1,"[windows, mac, linux]","[Single-player, Multi-player, Valve Anti-Cheat...",[Action],"[FPS, Action, Sci-fi]"
...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,1,[windows],"[Single-player, Steam Achievements]","[Adventure, Casual, Indie]","[Adventure, Indie, Casual]"
27071,1065570,Cyber Gun,1,[windows],[Single-player],"[Action, Adventure, Indie]","[Action, Indie, Adventure]"
27072,1065650,Super Star Blast,1,[windows],"[Single-player, Multi-player, Co-op, Shared/Sp...","[Action, Casual, Indie]","[Action, Indie, Casual]"
27073,1066700,New Yankee 7: Deer Hunters,1,"[windows, mac]","[Single-player, Steam Cloud]","[Adventure, Casual, Indie]","[Indie, Casual, Adventure]"


In [ ]:
def create_mlb_df(df, columns):

In [53]:
list(mlb.classes_)

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Documentary',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Gore',
 'Indie',
 'Massively Multiplayer',
 'Nudity',
 'Photo Editing',
 'RPG',
 'Racing',
 'Sexual Content',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Tutorial',
 'Utilities',
 'Video Production',
 'Violent',
 'Web Publishing']

In [54]:
split_genres_df = pd.DataFrame(genres_mlb, genres_df.index, mlb.classes_)
split_genres_df

,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Documentary,Early Access,Education,...,Sexual Content,Simulation,Software Training,Sports,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27071,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27072,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27073,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
clean_genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Sports',
 'Early Access']

In [58]:
len(steamspy_tags.keys())

339

## Testing Steam API Calls

In [1]:
!ls

README.md
SteamWebAPIKey.txt
data
notebook.ipynb
scratch_notebook.ipynb


In [93]:
key = './SteamWebAPIKey.txt'
with open(key, 'r') as file:
    key = file.read()

In [73]:
#my personal steam id
steam_id = 76561199032907135

In [60]:
f = open('./data/api_personal_account.json', 'r')
data = json.loads(f.read())
for game in data['response']['games']:
    print(game)
f.close()

{'appid': 6000, 'name': 'STAR WARSâ„¢ Republic Commando', 'playtime_forever': 291, 'img_icon_url': '73d4fd5437a5491479c7235eef64cf7d5e2c2e8e', 'playtime_windows_forever': 291, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1657320312, 'capsule_filename': 'portrait.png'}
{'appid': 32370, 'name': 'STAR WARSâ„¢: Knights of the Old Republicâ„¢', 'playtime_forever': 2746, 'img_icon_url': '018b6650c8de8c9c36d957cf6f8bc11d0c21d083', 'playtime_windows_forever': 2746, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1631947981}
{'appid': 6020, 'name': 'STAR WARSâ„¢ Jedi Knight: Jedi Academyâ„¢', 'playtime_forever': 473, 'img_icon_url': '2e359a8df71ea18ddd77fda3f6b307e6e86ef910', 'playtime_windows_forever': 473, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1657503557, 'capsule_filename': 'portrait.png', 'sort_as': 'Star Wars Jedi Knight 5'}
{'appid': 47810, 'name': 'Dragon Age: Origins - Ultimate Edition', 

In [61]:
f = open('./data/api_personal_account.json', 'r')
data = json.loads(f.read())
game_list = []
for game in data['response']['games']:
    game_list.append(game)
f.close()

In [62]:
game_list

[{'appid': 6000,
  'name': 'STAR WARSâ„¢ Republic Commando',
  'playtime_forever': 291,
  'img_icon_url': '73d4fd5437a5491479c7235eef64cf7d5e2c2e8e',
  'playtime_windows_forever': 291,
  'playtime_mac_forever': 0,
  'playtime_linux_forever': 0,
  'rtime_last_played': 1657320312,
  'capsule_filename': 'portrait.png'},
 {'appid': 32370,
  'name': 'STAR WARSâ„¢: Knights of the Old Republicâ„¢',
  'playtime_forever': 2746,
  'img_icon_url': '018b6650c8de8c9c36d957cf6f8bc11d0c21d083',
  'playtime_windows_forever': 2746,
  'playtime_mac_forever': 0,
  'playtime_linux_forever': 0,
  'rtime_last_played': 1631947981},
 {'appid': 6020,
  'name': 'STAR WARSâ„¢ Jedi Knight: Jedi Academyâ„¢',
  'playtime_forever': 473,
  'img_icon_url': '2e359a8df71ea18ddd77fda3f6b307e6e86ef910',
  'playtime_windows_forever': 473,
  'playtime_mac_forever': 0,
  'playtime_linux_forever': 0,
  'rtime_last_played': 1657503557,
  'capsule_filename': 'portrait.png',
  'sort_as': 'Star Wars Jedi Knight 5'},
 {'appid': 47

In [68]:
game_list[0]['name']

'STAR WARSâ„¢ Republic Commando'

In [ ]:
#sample format of json
{"response":
 {"game_count":44,
  "games":[{"appid":6000,"name":"STAR WARS™ Republic Commando",
            "playtime_forever":291,
            "img_icon_url":"73d4fd5437a5491479c7235eef64cf7d5e2c2e8e",
            "playtime_windows_forever":291,"playtime_mac_forever":0,
            "playtime_linux_forever":0,"rtime_last_played":1657320312,
            "capsule_filename":"portrait.png"},
           {"appid":32370,"name":"STAR WARS™: Knights of the Old Republic™",
            "playtime_forever":2746,
            "img_icon_url":"018b6650c8de8c9c36d957cf6f8bc11d0c21d083",
            "playtime_windows_forever":2746,
            "playtime_mac_forever":0,
            "playtime_linux_forever":0,
            "rtime_last_played":1631947981}
           
http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=&steamid=76561199032907135&include_appinfo=1&format=json

In [67]:
sw_rc = steam_df[steam_df['appid']==6000]
sw_rc['name']

147    STAR WARS™ Republic Commando™
Name: name, dtype: object

In [70]:
steam_df[steam_df['name'].str.contains(',')]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,percent_positive
133,4570,"Warhammer® 40,000: Dawn of War® - Game of the ...",2007-08-07,1,Relic Entertainment,SEGA,windows,16,Single-player;Multi-player;Steam Trading Cards...,Strategy,Strategy;Warhammer 40K;RTS,0,3485,206,309,340,1000000-2000000,9.99,3691,0.944189
134,4580,"Warhammer® 40,000: Dawn of War® - Dark Crusade",2007-08-07,1,Relic Entertainment,SEGA,windows,16,Single-player;Multi-player;Steam Cloud,Strategy,Strategy;Warhammer 40K;RTS,0,2849,133,543,620,1000000-2000000,9.99,2982,0.955399
152,6060,"Star Wars: Battlefront 2 (Classic, 2005)",2009-07-08,1,Pandemic Studios,Lucasfilm;LucasArts;Disney Interactive,windows,0,Single-player;Multi-player;Online Multi-Player...,Action,Star Wars;Action;Multiplayer,0,29149,2425,618,320,2000000-5000000,7.19,31574,0.923196
185,7210,"Runaway, A Road Adventure",2007-03-14,1,Pendulo Studios,Focus Home Interactive,windows,0,Single-player,Adventure,Adventure;Point & Click;Mystery,0,236,90,14,14,100000-200000,6.99,326,0.723926
186,7220,"Runaway, The Dream of The Turtle",2007-03-14,1,Pendulo Studios,Focus Home Interactive,windows,0,Single-player,Adventure,Adventure;Point & Click;Mystery,0,86,65,0,0,100000-200000,6.99,151,0.569536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26736,1035260,"A Game For You, Josh",2019-04-08,1,Penelope Evans,Penelope Evans,windows;mac,0,Single-player;Steam Achievements,Indie,Indie,6,1,0,0,0,0-20000,2.09,1,1.000000
26988,1053040,"Zom-bie, or Not Zom-bie",2019-04-17,1,John Michael O'Brien II;Games Master Mind,Games Master Mind,windows,0,Single-player;Steam Achievements,Indie;RPG,RPG;Indie;Action,25,2,1,0,0,0-20000,0.79,3,0.666667
27044,1058940,"Treasure Masters, Inc.: The Lost City",2019-04-02,1,Casa Games,Alawar Entertainment,windows,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Adventure;Indie;Casual,0,1,1,0,0,0-20000,5.19,2,0.500000
27055,1060770,"Die, zombie sausage, die!",2019-04-24,1,AuroraCorp,Aurora software,windows,0,Single-player;Steam Leaderboards,Action;Indie;Early Access,Early Access;Action;Indie,0,1,0,0,0,0-20000,0.79,1,1.000000


In [74]:
steam_id

76561199032907135

In [ ]:
76_561_199_032_907_135
76_561_198_045_426_389

In [112]:
#testing random steam ids to get library data
rng = np.random.RandomState(42)
random_ids = rng.randint(0, 999_000_000, size=1_000)
random_ids = [76_561_199_000_000_000 + i for i in random_ids]
random_ids

[76561199534895718,
 76561199199900595,
 76561199862061404,
 76561199787846414,
 76561199996406378,
 76561199127521863,
 76561199423734972,
 76561199415968276,
 76561199670094950,
 76561199841095289,
 76561199669991378,
 76561199429389014,
 76561199249467210,
 76561199898717130,
 76561199498972759,
 76561199359525748,
 76561199434285667,
 76561199613608295,
 76561199893664919,
 76561199648061058,
 76561199088409749,
 76561199242285876,
 76561199944505601,
 76561199953477463,
 76561199354088427,
 76561199809827741,
 76561199911989541,
 76561199003344769,
 76561199780932287,
 76561199787716372,
 76561199504579232,
 76561199232968651,
 76561199479546681,
 76561199106328085,
 76561199030349564,
 76561199781447915,
 76561199099052376,
 76561199177077808,
 76561199106406362,
 76561199480404538,
 76561199643647998,
 76561199599121577,
 76561199200427519,
 76561199181009883,
 76561199961022651,
 76561199499770319,
 76561199885063869,
 76561199389151677,
 76561199151079598,
 76561199508464061,


In [113]:
players = {}
for id in random_ids:
    api_call = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={key}&steamid={id}&include_appinfo=1&format=json'
    player_data = requests.get(api_call).json()
    players[id] = player_data

In [114]:
players

[{'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {'game_count': 2,
   'games': [{'appid': 730,
     'name': 'Counter-Strike: Global Offensive',
     'playtime_forever': 19629,
     'img_icon_url': '69f7ebe2735c366c65c0b33dae00e12dc40edbe4',
     'has_community_visible_stats': True,
     'playtime_windows_forever': 18559,
     'playtime_mac_forever': 0,
     'playtime_linux_forever': 0,
     'rtime_last_played': 1658218971},
    {'appid': 880940,
     'name': 'Pummel Party',
     'playtime_forever': 132,
     'img_icon_url': 'b2261da6c4f14808955181d9b41fdc242685a325',
     'has_community_visible_stats': True,
     'playtime_windows_forever': 132,
     'playtime_mac_forever': 0,
     'playtime_linux_forever': 0,
     'rtime_last_played': 1614804265}]}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {}},
 {'response': {'game_count': 1,
   'games': [{'appid': 730,
    

In [116]:
type(players[0])

dict

In [138]:
empty_dict = {}
players_dict = {}
for i in range(len(random_ids)):
    if players[i]['response'] != empty_dict:
        if players[i]['response']['game_count'] != 0:
            players_dict[random_ids[i]] = players[i]['response']
players_dict

{76561199127521863: {'game_count': 2,
  'games': [{'appid': 730,
    'name': 'Counter-Strike: Global Offensive',
    'playtime_forever': 19629,
    'img_icon_url': '69f7ebe2735c366c65c0b33dae00e12dc40edbe4',
    'has_community_visible_stats': True,
    'playtime_windows_forever': 18559,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'rtime_last_played': 1658218971},
   {'appid': 880940,
    'name': 'Pummel Party',
    'playtime_forever': 132,
    'img_icon_url': 'b2261da6c4f14808955181d9b41fdc242685a325',
    'has_community_visible_stats': True,
    'playtime_windows_forever': 132,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'rtime_last_played': 1614804265}]},
 76561199249467210: {'game_count': 1,
  'games': [{'appid': 730,
    'name': 'Counter-Strike: Global Offensive',
    'playtime_forever': 18,
    'img_icon_url': '69f7ebe2735c366c65c0b33dae00e12dc40edbe4',
    'has_community_visible_stats': True,
    'playtime_windows_forever': 18,
    

In [117]:
empty_dict={}
print(players[0])
players[0]['response'] == empty_dict

{'response': {}}


True

In [132]:
count=0
players_with_data = []
for player in players:
    if player['response'] != empty_dict:
        players_with_data.append(player)
        count += 1
print(count)

36


In [133]:
players_with_data

[{'response': {'game_count': 2,
   'games': [{'appid': 730,
     'name': 'Counter-Strike: Global Offensive',
     'playtime_forever': 19629,
     'img_icon_url': '69f7ebe2735c366c65c0b33dae00e12dc40edbe4',
     'has_community_visible_stats': True,
     'playtime_windows_forever': 18559,
     'playtime_mac_forever': 0,
     'playtime_linux_forever': 0,
     'rtime_last_played': 1658218971},
    {'appid': 880940,
     'name': 'Pummel Party',
     'playtime_forever': 132,
     'img_icon_url': 'b2261da6c4f14808955181d9b41fdc242685a325',
     'has_community_visible_stats': True,
     'playtime_windows_forever': 132,
     'playtime_mac_forever': 0,
     'playtime_linux_forever': 0,
     'rtime_last_played': 1614804265}]}},
 {'response': {'game_count': 1,
   'games': [{'appid': 730,
     'name': 'Counter-Strike: Global Offensive',
     'playtime_forever': 18,
     'img_icon_url': '69f7ebe2735c366c65c0b33dae00e12dc40edbe4',
     'has_community_visible_stats': True,
     'playtime_windows_forev